# *get_fence_array(patch_length, overall_length)* revisited
```python
def get_fence_array_from_padded_segment_with_stride(patch_len, start_seg, end_seg, fill_complete=False):
    
    return fence_array
```

## Compare padding vs stride in the context of registration intersection pairs

In [29]:
import numpy as np


def check_patch_in_bounds(x, y, X_dim, Y_dim):
    """ Usage: TrueFalse = check_patch_in_bounds(x, y, X_dim, Y_dim)
                determine if the box is within the image
    Args:
        x:               a tuple, list or array (x_start, x_end)
        y:               a tuple, list or array (y_start, Y_end)
        X_dim:           a tuple, list or array (Image_X_start, Image_X_end)
        Y_dim:           a tuple, list or array (Image_Y_start, Image_Y_end)
    """
    if x[0] > x[1] or y[0] > y[1] or X_dim[0] > X_dim[1] or Y_dim[0] > Y_dim[1]:
        return False
    
    if x[0] >= X_dim[0] and y[0] >= Y_dim[0] and x[1] < X_dim[1] and y[1] < Y_dim[1]:
        return True
    
    else:
        return False

    
def im_pair_hori(im_0, im_1):
    """ Usage: new_im = im_pair_hori(im_0, im_1)
            combine a list of PIL images horizontaly
    """
    w0 = im_0.size[0]
    w = w0 + im_1.size[0] + 1
    h = max(im_0.size[1], im_1.size[1])

    new_im = tip.Image.new('RGB', (w, h) )
    box = (0, 0, w0, h)
    new_im.paste(im_0, box)
    
    box = (w0+1, 0, w, h)
    new_im.paste(im_1, box)

    return new_im

X_dim, Y_dim = (120, 1193), (59, 1200)
x = (120 + 25, 344)
y = (60+280, 284)
print(x, y)
x = list(x)
y = np.array(y)
check_patch_in_bounds(x, y, X_dim, Y_dim)

(145, 344) (340, 284)


False

In [15]:
x = (0, 10)
X = (0, 10)
x and X

(0, 10)

In [7]:
def get_strided_fence_array_NB(patch_length, start_seg, end_seg):
    """ Usage: fence_array = get_fence_array(patch_length, overall_length)
        create a left-right set of pairs that descrete overall_length into patch_length segments

    Args:
        patch_length:   patch size - number of pixels high or wide
        patch_length:   overall number of pixels high or wide

    Returns:
        fence_array:    boundry values for each segment
        -----------:    [[left_0, right_0],
                         [left_1, right_1],
                         [left_2, right_2],... ]
    """
    #                               determine the stride:
    overall_length = end_seg - start_seg
    n_fenced = np.ceil(overall_length / patch_length).astype(int)  # number of boxes
    stride = np.ceil(overall_length / n_fenced).astype(int)
    
    #                               initialize the return array:
    fence_array = np.zeros((n_fenced, 2)).astype(int)
    
    #                               fill in the beginning pair:
    fence_array[0,0] = start_seg
    fence_array[0,1] = start_seg + patch_length - 1
    
    #                               fill in the rest, including the last:
    for k in range(1, fence_array.shape[0]):    
        fence_array[k, 0] = fence_array[k-1, 0] + stride
        fence_array[k, 1] = fence_array[k, 0] + patch_length - 1
        
    #                               re-fill in the last one measured from the end:
    fence_array[k, 1] = end_seg - 1
    fence_array[k, 0] = end_seg - patch_length
    
    return fence_array

patch_length = 224
remainder = 23
overall_length = 224*12 + remainder
start_seg = 22
end_seg = overall_length - 211

print('patch_length', patch_length, 'start_segment', start_seg, 'end_segment %i\n'%(end_seg))
fa = get_strided_fence_array_NB(patch_length, start_seg, end_seg)
print('Array Size =  %i\n'%(len(fa)))
for f in fa:
    print(f, '\t%i'%(f[1] - f[0]))


patch_length 224 start_segment 22 end_segment 2500

overall_length: 2478
stride 207 n_fenced * stride 2484
Array Size =  12

[ 22 245] 	223
[229 452] 	223
[436 659] 	223
[643 866] 	223
[ 850 1073] 	223
[1057 1280] 	223
[1264 1487] 	223
[1471 1694] 	223
[1678 1901] 	223
[1885 2108] 	223
[2092 2315] 	223
[2276 2499] 	223


```sh
# patch_length 224 overall_length 2711 remainder 23

[ 12 235]
[236 459]
[460 683]
[684 907]
[ 908 1131]
[1132 1355]
[1356 1579]
[1580 1803]
[1804 2027]
[2028 2251]
[2252 2475]
[2476 2699]
```

In [3]:
# patch_length 224 overall_length 2711 remainder 23

def get_fence_array_NB(patch_length, overall_length):
    """ Usage: fence_array = get_fence_array(patch_length, overall_length)
        create a left-right set of pairs that descrete overall_length into patch_length segments

    Args:
        patch_length:   patch size - number of pixels high or wide
        patch_length:   overall number of pixels high or wide

    Returns:
        fence_array:    boundry values for each segment
        -----------:    [[left_0, right_0],
                         [left_1, right_1],
                         [left_2, right_2],... ]
    """
    # Determine the array size
    n_fenced = overall_length // patch_length  # number of boxes
    n_remain = 1 + overall_length % patch_length  # number of pixels leftover
    paddit = n_remain // 2  # padding for the beginning

    if n_remain == patch_length:  # exact fit special case: exactly one left over
        paddit = 0
        n_fenced = n_fenced + 1

    # Allocate as integers for use as indices
    fence_array = np.zeros((n_fenced, 2)).astype(int)
    for k in range(n_fenced):
        # for each box edge, get the beginning and end pixel location
        if k == 0:
            # first case special (padding)
            fence_array[k, 0] = paddit
            # add the width to it
            fence_array[k, 1] = fence_array[k, 0] + patch_length - 1

        elif fence_array[k - 1, 1] + patch_length <= overall_length:
            # Previous right pixel plus one
            fence_array[k, 0] = fence_array[k - 1, 1] + 1
            # add the width to it
            fence_array[k, 1] = fence_array[k, 0] + patch_length - 1

    return fence_array

print('patch_length', patch_length, 'overall_length', overall_length, 'remainder %i\n'%(remainder))
fa_old = get_fence_array_NB(patch_length, overall_length)
print('Array Size =  %i\n'%(len(fa_old)))
for f0 in fa_old:
    print(f0, '\t%i'%(f0[1] - f0[0]))


patch_length 224 overall_length 2711 remainder 23

Array Size =  12

[ 12 235] 	223
[236 459] 	223
[460 683] 	223
[684 907] 	223
[ 908 1131] 	223
[1132 1355] 	223
[1356 1579] 	223
[1580 1803] 	223
[1804 2027] 	223
[2028 2251] 	223
[2252 2475] 	223
[2476 2699] 	223


In [37]:
import time
nb_start_time = time.time()

import os
import tempfile
import sys

import numpy as np
import pandas as pd

import openslide

sys.path.insert(0, '../src/python')
from digipath_toolkit import get_level_sizes_dict, get_sample_selection_mask

In [34]:
test_data_dir = '../../DigiPath_MLTK_data/RegistrationDevData/'
os.listdir(test_data_dir)

offset_data_file = os.path.join(test_data_dir, 'wsi_pair_sample.csv')
if os.path.isfile(offset_data_file):
    offset_df = pd.read_csv(offset_data_file)
    
offset_x = offset_df['truth_offset_x'].iloc[0]
offset_y = offset_df['truth_offset_y'].iloc[0]
offset_x, offset_y = int(round(offset_x)), int(round(offset_y))

auto_x = offset_df['auto_offset_x'].iloc[0]
auto_y = offset_df['auto_offset_y'].iloc[0]
auto_x, auto_y = int(round(auto_x)), int(round(auto_y))

print('\noffset_x, offset_y, auto_x, auto_y\n', offset_x, offset_y, auto_x, auto_y)

fixed_wsi = os.path.join(test_data_dir, '54742d6c5d704efa8f0814456453573a.tiff')

fixed_levels_dict = get_level_sizes_dict(fixed_wsi)
fixed_max_width = fixed_levels_dict['image_size'][0]
fixed_max_height = fixed_levels_dict['image_size'][1]

print('\n\nfixed_wsi image: 54742d6c5d704efa8f0814456453573a.tiff')
for k, v in fixed_levels_dict.items():
    print('%25s: %s'%(k,v))
    
float_wsi = os.path.join(test_data_dir, 'e39a8d60a56844d695e9579bce8f0335.tiff')

float_levels_dict = get_level_sizes_dict(float_wsi)
float_max_width = float_levels_dict['image_size'][0]
float_max_height = float_levels_dict['image_size'][1]

print('\n\nfloat_wsi image: e39a8d60a56844d695e9579bce8f0335.tiff')
for k, v in float_levels_dict.items():
    print('%25s: %s'%(k,v))
    
run_parameters = {'wsi_filename': fixed_wsi, 
                  'wsi_floatname': float_wsi,
                  'thumbnail_divisor': 20, 
                  'patch_select_method': 'threshold_rgb2lab', 
                  'rgb2lab_threshold': 80, 
                  'image_level': 0, 
                  'patch_height': 224, 
                  'patch_width': 224, 
                  'threshold': 0, 
                  'offset_x': offset_x, 
                  'offset_y': offset_y}


offset_x, offset_y, auto_x, auto_y
 -1618 1673 -1621 1676


fixed_wsi image: 54742d6c5d704efa8f0814456453573a.tiff
               image_size: (145408, 83968)
              level_count: 10
        level_downsamples: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0)
         level_diminsions: ((145408, 83968), (72704, 41984), (36352, 20992), (18176, 10496), (9088, 5248), (4544, 2624), (2272, 1312), (1136, 656), (568, 328), (284, 164))


float_wsi image: e39a8d60a56844d695e9579bce8f0335.tiff
               image_size: (126976, 75776)
              level_count: 9
        level_downsamples: (1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0)
         level_diminsions: ((126976, 75776), (63488, 37888), (31744, 18944), (15872, 9472), (7936, 4736), (3968, 2368), (1984, 1184), (992, 592), (496, 296))


In [38]:
def get_patch_location_array_for_image_level_NB(run_parameters):
    """ Usage: patch_location_array = get_patch_location_array_for_image_level(run_parameters)
        using 'patch_select_method", find all upper left corner locations of patches
        that won't exceed image size givin the 'patch_height' and 'patch_width'

    Args (run_parameters):  python dict.keys()
                                wsi_filename:           file name (with valid path)
                                patch_height:           patch size = (patch_width, patch_height)
                                patch_width:            patch size = (patch_width, patch_height)
                                thumbnail_divisor:      wsi_image full size divisor to create thumbnail image
                                patch_select_method:    'threshold_rgb2lab' or 'threshold_otsu'
                                threshold:              minimimum sum of thresholded image (default = 0)
                                image_level:            openslide image pyramid level 0,1,2,...
    Returns:
        patch_location_array

    """
    #                   initialize an empty return value
    patch_location_array = []
    #                   name the input variables
    wsi_filename = run_parameters['wsi_filename']
    thumbnail_divisor = run_parameters['thumbnail_divisor']
    patch_select_method = run_parameters['patch_select_method']
    patch_height = run_parameters['patch_height']
    patch_width = run_parameters['patch_width']
    #                   set defaults for newly added parameters
    if 'threshold' in run_parameters:
        threshold = run_parameters['threshold']
    else:
        threshold = 0

    if 'image_level' in run_parameters:
        image_level = run_parameters['image_level']
    else:
        image_level = 0

    #                     OpenSlide open                      #
    os_im_obj = openslide.OpenSlide(wsi_filename)
    obj_level_diminsions = os_im_obj.level_dimensions

    #                   get the start, stop locations list for the rows
    pixels_height = obj_level_diminsions[image_level][1]
    start_row = 0
    rows_fence_array = get_strided_fence_array_NB(patch_height, start_row, pixels_height)

    #                   get the start, stop locations list for the columns
    pixels_width = obj_level_diminsions[image_level][0]
    start_col = 0
    cols_fence_array = get_strided_fence_array_NB(patch_width, start_col, pixels_width)

    #                   get a thumbnail image for the patch select method
    thumbnail_size = (pixels_width // thumbnail_divisor, pixels_height // thumbnail_divisor)
    small_im = os_im_obj.get_thumbnail(thumbnail_size)
    os_im_obj.close()
    #                     OpenSlide close                     #

    #                   get the binary mask as a measure of image region content
    mask_im = get_sample_selection_mask(small_im, patch_select_method)

    #                   iterator for rows:  (top_row, bottom_row, full_scale_row_number)
    it_rows = zip(rows_fence_array[:, 0] // thumbnail_divisor,
                  rows_fence_array[:, 1] // thumbnail_divisor,
                  rows_fence_array[:, 0])

    #                   variables for columns iterator
    lft_cols = cols_fence_array[:, 0] // thumbnail_divisor
    rgt_cols = cols_fence_array[:, 1] // thumbnail_divisor
    cols_array = cols_fence_array[:, 0]

    for tmb_row_top, tmb_row_bot, row_n in it_rows:
        #               iterator for cols:  (left_column, right_column, full_scale_column_number)
        it_cols = zip(lft_cols, rgt_cols, cols_array)

        for tmb_col_lft, tmb_col_rgt, col_n in it_cols:

            #           if the sum of the mask elements is larger than the threshold...
            if (mask_im[tmb_row_top:tmb_row_bot, tmb_col_lft:tmb_col_rgt]).sum() > threshold:

                #       add the full scale row and column of the upper left corner to the list
                patch_location_array.append((row_n, col_n))

    return patch_location_array

patch_location_array = get_patch_location_array_for_image_level_NB(run_parameters)

len(patch_location_array)

overall_length: 83968
stride 224 n_fenced * stride 84000
overall_length: 145408
stride 224 n_fenced * stride 145600


55966

In [42]:
for k, v in run_parameters.items():
    print(k,v)
    
print('\n%20s:'%('key_name'))
print('(fixed values)')
print('(float values)')

for k, v in fixed_levels_dict.items():
    print('\n%20s:'%(k))
    print(v)
    print(float_levels_dict[k])
    


wsi_filename ../../DigiPath_MLTK_data/RegistrationDevData/54742d6c5d704efa8f0814456453573a.tiff
wsi_floatname ../../DigiPath_MLTK_data/RegistrationDevData/e39a8d60a56844d695e9579bce8f0335.tiff
thumbnail_divisor 20
patch_select_method threshold_rgb2lab
rgb2lab_threshold 80
image_level 0
patch_height 224
patch_width 224
threshold 0
offset_x -1618
offset_y 1673

            key_name:
(fixed values)
(float values)

          image_size:
(145408, 83968)
(126976, 75776)

         level_count:
10
9

   level_downsamples:
(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0, 512.0)
(1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0)

    level_diminsions:
((145408, 83968), (72704, 41984), (36352, 20992), (18176, 10496), (9088, 5248), (4544, 2624), (2272, 1312), (1136, 656), (568, 328), (284, 164))
((126976, 75776), (63488, 37888), (31744, 18944), (15872, 9472), (7936, 4736), (3968, 2368), (1984, 1184), (992, 592), (496, 296))
